In [1]:
import geopandas as gpd

In [5]:
traffico = gpd.read_file('./resources/traffico/traffico.shp').to_crs(epsg=4326)

In [6]:
traffico

,desc_zona,id_zona,sigla_prov,geometry
0,PAVIA 1,1435.0,PV,"MULTIPOLYGON (((9.13877 45.18912, 9.13870 45.1..."
1,PAVIA 2,1436.0,PV,"POLYGON ((9.11517 45.18448, 9.11544 45.18444, ..."
2,ISSO,577.0,BG,"POLYGON ((9.76190 45.48423, 9.76136 45.48418, ..."
3,PORTALBERA,930.0,PV,"POLYGON ((9.30362 45.10433, 9.30412 45.10425, ..."
4,CREMONA 1,1439.0,CR,"POLYGON ((10.01167 45.14215, 10.01270 45.14215..."
...,...,...,...,...
1445,CA` D`ANDREA,982.0,CR,"POLYGON ((10.30939 45.09905, 10.30894 45.09917..."
1446,CAPRALBA,987.0,CR,"POLYGON ((9.66830 45.41695, 9.66845 45.41596, ..."
1447,CASALBUTTANO ED UNITI,988.0,CR,"POLYGON ((9.99783 45.24696, 9.99834 45.24688, ..."
1448,CASALETTO CEREDANO,989.0,CR,"POLYGON ((9.63481 45.31477, 9.63471 45.31464, ..."


In [8]:
traffico['Centroide'] = traffico['geometry'].to_crs(epsg=4326).centroid

<ipython-input-8-26c61d1c4e94>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  traffico['Centroide'] = traffico['geometry'].to_crs(epsg=4326).centroid


In [15]:
traffico.drop('geometry', axis=1, inplace=True)

In [39]:
traffico['Latitude'] = traffico['Centroide'].apply(lambda x: x.coords[0][1])
traffico['Longitude'] = traffico['Centroide'].apply(lambda x: x.coords[0][0])

In [41]:
traffico.to_csv('./resources/traffico/traffico_with_centroide.csv')

In [43]:
import pandas as pd

trasporti = pd.read_csv('./data/transports_compressed.csv')

In [49]:
unique_zones = list(set(traffico['desc_zona'].values))

In [55]:
trasporti['LAT_ORIG'] = 0
trasporti['LON_ORIG'] = 0
trasporti['LAT_DEST'] = 0
trasporti['LON_DEST'] = 0

In [65]:
zones = {}

for zona in unique_zones:

    t = traffico[traffico['desc_zona'] == zona]
    lat = t['Latitude'].values[0]
    lon = t['Longitude'].values[0]

    zones.update({zona: {'lat': lat, 'lon': lon}})

In [80]:
trasporti['LAT_ORIG'] = trasporti['ZONA_ORIG'].apply(lambda zone: zones[zone]['lat'])
trasporti['LON_ORIG'] = trasporti['ZONA_ORIG'].apply(lambda zone: zones[zone]['lon'])
trasporti['LAT_DEST'] = trasporti['ZONA_DEST'].apply(lambda zone: zones[zone]['lat'])
trasporti['LON_DEST'] = trasporti['ZONA_DEST'].apply(lambda zone: zones[zone]['lon'])

In [72]:
idxs = trasporti.index[(~trasporti['ZONA_ORIG'].isin(unique_zones)) | (~trasporti['ZONA_DEST'].isin(unique_zones))].tolist()

In [74]:
trasporti.drop(index=idxs, inplace=True)

In [79]:
zones['ABBADIA CERRETO - CORTE PALASIO']

{'lat': 45.31290973698156, 'lon': 9.575083193581557}

In [87]:
trasporti.to_csv('./data/trasporti_with_coords.csv')

In [83]:
trasporti.head()

,Unnamed: 0,ZONA_ORIG,ZONA_DEST,LAV_COND,LAV_MOTO,LAV_GOMMA,STU_COND,STU_MOTO,STU_GOMMA,OCC_COND,...,AFF_MOTO,AFF_GOMMA,RIT_COND,RIT_MOTO,RIT_GOMMA,Total,LAT_ORIG,LON_ORIG,LAT_DEST,LON_DEST
0,0,ABBADIA CERRETO - CORTE PALASIO,ABBADIA CERRETO - CORTE PALASIO,69.72,5.56,1.07,0.0,0.0,1.86,143.89,...,0.0,0.0,169.26,9.43,4.84,419.14,45.31291,9.575083,45.312910,9.575083
1,1,ABBADIA CERRETO - CORTE PALASIO,ALBESE CON CASSANO,0.00,0.00,0.00,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,0.03,0.00,0.03,45.31291,9.575083,45.808834,9.163915
2,2,ABBADIA CERRETO - CORTE PALASIO,BAGNOLO CREMASCO,13.85,0.00,0.54,0.0,0.0,0.58,31.21,...,0.0,0.0,18.27,0.00,9.23,85.07,45.31291,9.575083,45.365271,9.613618
4,4,ABBADIA CERRETO - CORTE PALASIO,BERTONICO,0.60,0.00,0.00,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.60,45.31291,9.575083,45.245437,9.671625
6,6,ABBADIA CERRETO - CORTE PALASIO,BORGHETTO LODIGIANO,0.60,0.00,0.00,0.0,0.0,0.00,0.00,...,0.0,0.0,0.53,0.00,0.00,1.13,45.31291,9.575083,45.211719,9.498890


In [84]:
idxs = trasporti.index[trasporti['Total'] < 100].tolist()

In [85]:
trasporti.drop(index=idxs, inplace=True)

In [86]:
len(trasporti)

16126